# Git Analysis
This notebook contains ways of visualizing git repositories in ways that are separate than the files in them. This is focused on visualizing the flow of commits, contributors, size of commits, etc. over time.

In [2]:
# Install statements for things not present in Anaconda
%pip install plotly
%pip install pydriller

Note: you may need to restart the kernel to use updated packages.
  Using cached PyDriller-2.0-py3-none-any.whl (65 kB)
  Using cached lizard-1.17.9-py2.py3-none-any.whl (64 kB)
  Using cached gitdb-4.0.7-py3-none-any.whl (63 kB)
  Using cached smmap-4.0.0-py2.py3-none-any.whl (24 kB)
Note: you may need to restart the kernel to use updated packages.


In [17]:
# Imports

from pydriller import Repository

import pandas as pd
import plotly.express as px

account = 'IntegerMan'
repository = 'VisualizingCode'

path = 'https://github.com/' + account + '/' + repository

print('Using repository ' + path)

Using repository https://github.com/IntegerMan/VisualizingCode


In [41]:
repo = Repository(path)

commits = []

for commit in repo.traverse_commits():
    record = {
        'hash': commit.hash,
        'message': commit.msg,
        'author_name': commit.author.name,
        'author_email': commit.author.email,
        'author_date': commit.author_date,
        'author_tz': commit.author_timezone,
        'committer_name': commit.committer.name,
        'committer_email': commit.committer.email,
        'committer_date': commit.committer_date,
        'committer_tz': commit.committer_timezone,
        'in_main': commit.in_main_branch,
        'is_merge': commit.merge,
        'num_deletes': commit.deletions,
        'num_inserts': commit.insertions,
        'net_lines': commit.lines,
        'num_files': commit.files,
        'branches': ', '.join(commit.branches), # Comma separated list of branches the commit is found in
        #'files': ', '.join(commit.modified_files), # Comma separated list of files the commit modifies
        'parents': ', '.join(commit.parents), # Comma separated list of parents
        # PyDriller Open Source Delta Maintainability Model (OS-DMM) stat. See https://pydriller.readthedocs.io/en/latest/deltamaintainability.html for metric definitions
        'dmm_unit_size': commit.dmm_unit_size,
        'dmm_unit_complexity': commit.dmm_unit_complexity,
        'dmm_unit_interfacing': commit.dmm_unit_interfacing,
    }
    # Omitted: modified_files (list), parents(list), project_path, project_name
    commits.append(record)

In [43]:
df_commits = pd.DataFrame(commits)
df_commits.to_csv('RawCommits.csv')
df_commits.head()

,hash,message,author_name,author_email,author_date,author_tz,committer_name,committer_email,committer_date,committer_tz,...,is_merge,num_deletes,num_inserts,net_lines,num_files,branches,parents,dmm_unit_size,dmm_unit_complexity,dmm_unit_interfacing
0,262f4fbfa31cc7e1d9bb7b9871be1c9ac0242c57,Initial commit,Matt Eland,Matt.Eland@GMail.com,2021-10-07 00:50:53-04:00,14400,GitHub,noreply@github.com,2021-10-07 00:50:53-04:00,14400,...,False,0,131,131,2,main,,NaN,NaN,NaN
1,436cece5340eb314fe5fe0eef94e423388755b23,Added code analysis export results from Visual...,Matt Eland,Matt.Eland@GMail.com,2021-10-07 00:53:05-04:00,14400,Matt Eland,Matt.Eland@GMail.com,2021-10-07 00:53:05-04:00,14400,...,False,0,0,0,1,main,262f4fbfa31cc7e1d9bb7b9871be1c9ac0242c57,NaN,NaN,NaN
2,1b7c69a77563a9845cee2e5a55affea4c590bf17,Early work on file mining,Matt Eland,Matt.Eland@GMail.com,2021-10-07 14:33:19-04:00,14400,Matt Eland,Matt.Eland@GMail.com,2021-10-07 14:33:19-04:00,14400,...,False,0,109,109,9,main,436cece5340eb314fe5fe0eef94e423388755b23,1.0,1.0,1.0
3,5d59ceb5390057755c3648307324e30d5ea0dc92,Mining Progress,Matt Eland,Matt.Eland@GMail.com,2021-10-07 17:39:49-04:00,14400,Matt Eland,Matt.Eland@GMail.com,2021-10-07 17:39:49-04:00,14400,...,False,32,217,249,3,main,1b7c69a77563a9845cee2e5a55affea4c590bf17,1.0,1.0,1.0
4,c745f4f151247070e3d5ee9c3467cd3dec763cfd,Additional Mining,Matt Eland,Matt.Eland@GMail.com,2021-10-07 19:05:52-04:00,14400,Matt Eland,Matt.Eland@GMail.com,2021-10-07 19:05:52-04:00,14400,...,False,3,3,6,1,main,5d59ceb5390057755c3648307324e30d5ea0dc92,NaN,NaN,NaN


In [32]:
df_commits.describe()

,author_tz,committer_tz,num_deletes,num_inserts,net_lines,num_files,dmm_unit_size,dmm_unit_complexity,dmm_unit_interfacing
count,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,6.000000,6.000000,6.000000
mean,10800.000000,10800.000000,76.375000,1518.541667,1594.916667,2.250000,0.500000,0.725490,0.833333
std,6369.492506,6369.492506,337.808643,4610.281808,4890.935599,2.231202,0.547723,0.439671,0.408248
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,10800.000000,10800.000000,0.000000,1.000000,2.000000,1.000000,0.000000,0.514706,1.000000
50%,14400.000000,14400.000000,1.000000,1.500000,4.500000,1.000000,0.500000,1.000000,1.000000
75%,14400.000000,14400.000000,5.500000,79.000000,100.000000,2.250000,1.000000,1.000000,1.000000
max,14400.000000,14400.000000,1661.000000,19032.000000,20693.000000,9.000000,1.000000,1.000000,1.000000


In [44]:
# Histogram of net lines of code
fig = px.histogram(df_commits,
                   x="net_lines",
                   title='Net Lines of Code',
                   labels={'net_lines': 'Net Lines of Code'},
                   color_discrete_sequence=px.colors.qualitative.Dark24)
fig.show()

In [45]:
# Histogram of net lines of code
fig = px.scatter(df_commits, x='num_files', y='net_lines', hover_name='message', hover_data=['author_name', 'author_date', 'hash', 'branches'])
fig.show()